In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('linearReg').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression
df = spark.read.csv('FileStore/tables/Ecommerce_Customers.csv', inferSchema=True, header=True)

In [4]:
df.printSchema()

root
-- Email: string (nullable = true)
-- Address: string (nullable = true)
-- Avatar: string (nullable = true)
-- Avg Session Length: double (nullable = true)
-- Time on App: double (nullable = true)
-- Time on Website: double (nullable = true)
-- Length of Membership: double (nullable = true)
-- Yearly Amount Spent: double (nullable = true)

In [5]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [6]:
df.columns

Out[ 6 ]: 
['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [7]:
assembler = VectorAssembler(inputCols=['Avg Session Length',
                                       'Time on App', 
                                       'Time on Website',
                                       'Length of Membership'],
                            outputCol='features')

In [8]:
output = assembler.transform(df)

In [9]:
output.printSchema()

root
-- Email: string (nullable = true)
-- Address: string (nullable = true)
-- Avatar: string (nullable = true)
-- Avg Session Length: double (nullable = true)
-- Time on App: double (nullable = true)
-- Time on Website: double (nullable = true)
-- Length of Membership: double (nullable = true)
-- Yearly Amount Spent: double (nullable = true)
-- features: vector (nullable = true)

In [10]:
output.head(1)

Out[ 10 ]: [Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [11]:
final_data = output.select(['features', 'Yearly Amount Spent'])

In [12]:
final_data.show()

+--------------------+-------------------+
 features|Yearly Amount Spent|
+--------------------+-------------------+
[34.4972677251122...| 587.9510539684005|
[31.9262720263601...| 392.2049334443264|
[33.0009147556426...| 487.54750486747207|
[34.3055566297555...| 581.8523440352177|
[33.3306725236463...| 599.4060920457634|
[33.8710378793419...| 637.102447915074|
[32.0215955013870...| 521.5721747578274|
[32.7391429383803...| 549.9041461052942|
[33.9877728956856...| 570.2004089636196|
[31.9365486184489...| 427.1993848953282|
[33.9925727749537...| 492.6060127179966|
[33.8793608248049...| 522.3374046069357|
[29.5324289670579...| 408.6403510726275|
[33.1903340437226...| 573.4158673313865|
[32.3879758531538...| 470.4527333009554|
[30.7377203726281...| 461.7807421962299|
[32.1253868972878...| 457.84769594494855|
[32.3388993230671...| 407.70454754954415|
[32.1878120459321...| 452.3156754800354|
[32.6178560628234...| 605.061038804892|
+--------------------+-------------------+
only showing top 20 rows

In [13]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [14]:
train_data.describe().show()

+-------+-------------------+
summary|Yearly Amount Spent|
+-------+-------------------+
 count| 347|
 mean| 502.6746264918063|
 stddev| 79.90870220171965|
 min| 256.67058229005585|
 max| 765.5184619388373|
+-------+-------------------+

In [15]:
test_data.describe().show()

+-------+-------------------+
summary|Yearly Amount Spent|
+-------+-------------------+
 count| 153|
 mean| 491.69231200417295|
 stddev| 77.67048615982084|
 min| 298.76200786180766|
 max| 744.2218671047146|
+-------+-------------------+

In [16]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [17]:
lr_model = lr.fit(train_data)

In [18]:
test_result = lr_model.evaluate(test_data)

In [19]:
test_result.residuals.show()

+-------------------+
 residuals|
+-------------------+
-11.510830807364414|
 10.609161688151858|
-3.9098502918024565|
 5.42554966685077|
 -12.53660529775101|
 22.15809760585671|
 4.399790386421387|
 -6.414540510904203|
 -4.816415455837387|
-17.884869096997136|
-1.7468347531843165|
 18.244724849150316|
 6.640597912908959|
 -26.2067566415011|
-1.6860707960324248|
 -6.480423797921446|
-10.821259221064565|
-3.4008336994752426|
 -18.37966309389691|
-1.9381773219803904|
+-------------------+
only showing top 20 rows

In [20]:
test_result.rootMeanSquaredError

Out[ 20 ]: 10.476181910006893

In [21]:
test_result.r2

Out[ 21 ]: 0.9816877437153712

In [22]:
final_data.describe().show()

+-------+-------------------+
summary|Yearly Amount Spent|
+-------+-------------------+
 count| 500|
 mean| 499.3140382585909|
 stddev| 79.3147815497068|
 min| 256.67058229005585|
 max| 765.5184619388373|
+-------+-------------------+

In [23]:
pred_data = test_data.select('features')

In [24]:
predictions = lr_model.transform(pred_data)

In [25]:
predictions.show()

+--------------------+------------------+
 features| prediction|
+--------------------+------------------+
[30.3931845423455...| 331.439700610558|
[30.7377203726281...| 451.171580508078|
[30.8794843441274...|494.11645027665713|
[31.0472221394875...| 387.0718495221706|
[31.0662181616375...|461.46989850542536|
[31.2834474760581...| 569.6229918198107|
[31.3662121671876...|426.18909217006353|
[31.4474464941278...| 425.0172826061282|
[31.5147378578019...| 494.6289034522988|
[31.5702008293202...| 563.830361238402|
[31.5761319713222...| 542.9734187425126|
[31.6005122003032...| 460.9281266419466|
[31.6548096756927...|468.62282581463955|
[31.6739155032749...| 501.9318245513823|
[31.7216523605090...|349.46299742790507|
[31.7242025238451...|509.86831108588194|
[31.8093003166791...| 547.5931585839057|
[31.8124825597242...|396.21117868327246|
[31.8164283341993...| 519.5021545975533|
[31.8186165667690...|448.35685069211604|
+--------------------+------------------+
only showing top 20 rows